<a href="https://colab.research.google.com/github/akritidu/Multi-label-Classification-in-the-Home-Improvement/blob/main/LSTM_Bi_LSTM_GRU_Bi_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Uploading and Cleaning 

In [7]:

from numpy import array
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM, GRU, Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D, SpatialDropout1D, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate

from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import Dense, Input, LSTM, SimpleRNN, Embedding, Dropout, SpatialDropout1D, Activation, Conv1D,GRU
from tensorflow.keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization, Add, Flatten
from tensorflow.keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import tensorflow as tf
import re

# from tensorflow.keras.models import Sequential


In [2]:
df = pd.read_csv('Preprocessed_dataset.csv') #actual
train_df = df
train_df

,contexts,ham,phishing
0,google window google bypjtqaxxnab getei functi...,1,0
1,function envflush function window require requ...,1,0
2,timing timing timing tick function label time ...,1,0
3,yahoo deutschland roundtrip rttop number date ...,1,0
4,html overflow-y auto body font arial text-alig...,1,0
5,wikipedia unicode-bidi embed white-space nowra...,1,0
6,dtfirstbyte date meta http-equiv refresh conte...,1,0
7,blogger erstellen kostenloses blog html body t...,1,0
8,twitter document domain twitter copy twttr app...,1,0
9,body margin pad font-size line-height font-fam...,1,0


In [3]:
categories = df[['ham', 'phishing']]
categories.head()

,ham,phishing
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [4]:
## for single output layer
X  = df['contexts'] 
y = categories.values

In [5]:
tokenizer = Tokenizer(num_words=3000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
vocab_size = len(tokenizer.word_index) + 1
maxlen = 200
X = pad_sequences(X, padding='post', maxlen=maxlen)


In [6]:
from numpy import array
from numpy import asarray
from numpy import zeros


embeddings_dictionary = dict()

glove_file = open("../Datasets/glove.6B.50d.txt", encoding="utf-8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 50))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

# BiGRU Model

In [9]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import tensorflow as tf
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 50, weights=[embedding_matrix], trainable=False)(deep_inputs)
model = SpatialDropout1D(0.2)(embedding_layer)
model = Bidirectional(GRU(64,return_sequences=True,dropout=0.1,recurrent_dropout=0.1 ))(model)
model = Conv1D(128, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(model)
avg_pool = GlobalAveragePooling1D()(model)
max_pool = GlobalMaxPooling1D()(model)
model = concatenate([avg_pool, max_pool])
dense_layer1 = Dense(2, activation="sigmoid")(model)
Bi_GRU_glove = Model(deep_inputs, dense_layer1)
Bi_GRU_glove.compile(loss='binary_crossentropy',optimizer='adam',metrics=[tf.keras.metrics.AUC()])

In [10]:
import datetime
print(datetime.datetime.now())
kf = KFold(n_splits=5, shuffle=True)

accuracy_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    Bi_GRU_glove.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)
    y_pred = np.argmax(Bi_GRU_glove.predict(X_test), axis=-1)
    accuracy = accuracy_score(np.argmax(y_test, axis=-1), y_pred)
    print(accuracy)
    accuracy_scores.append(accuracy)
    
mean_accuracy = np.mean(accuracy_scores)
print('Mean Accuracy:', mean_accuracy)
print(datetime.datetime.now())

2023-09-10 04:04:00.330856
Epoch 1/5
122/122 [==============================] - 21s 174ms/step - loss: 0.2688 - auc_1: 0.9553
Epoch 2/5
122/122 [==============================] - 22s 179ms/step - loss: 0.1989 - auc_1: 0.9746
Epoch 3/5
122/122 [==============================] - 22s 179ms/step - loss: 0.1872 - auc_1: 0.9778
Epoch 4/5
122/122 [==============================] - 22s 179ms/step - loss: 0.1696 - auc_1: 0.9811
Epoch 5/5
122/122 [==============================] - 22s 179ms/step - loss: 0.1594 - auc_1: 0.9837
0.9406041986687148
Epoch 1/5
123/123 [==============================] - 32s 258ms/step - loss: 0.1596 - auc_1: 0.9837
Epoch 2/5
123/123 [==============================] - 31s 253ms/step - loss: 0.1574 - auc_1: 0.9839
Epoch 3/5
123/123 [==============================] - 31s 253ms/step - loss: 0.1466 - auc_1: 0.9861
Epoch 4/5
123/123 [==============================] - 31s 253ms/step - loss: 0.1354 - auc_1: 0.9881
Epoch 5/5
123/123 [==============================] - 31s 253ms/

In [11]:
Bi_GRU_glove.save('NDLP_model.h5')